## 1- Control and adjoint on kernel function

- #### Control on kernel function:

In [1]:
import cudaq

cudaq.set_target('nvidia')

@cudaq.kernel
def fancyCnot(a: cudaq.qubit, b: cudaq.qubit):
    x.ctrl(a, b)

@cudaq.kernel
def toffoli():
    q = cudaq.qvector(3)
    ctrl = q.front()
    # without a control, apply x to all
    x(ctrl, q[2])
    cudaq.control(fancyCnot, [ctrl], q[1], q[2])

counts = cudaq.sample(toffoli)

print(counts)

{ 101:1000 }



- #### Adjoint on kernel function:

In [2]:
import cudaq

cudaq.set_target('nvidia')

@cudaq.kernel()
def rx_and_h_gate(x : float, q : cudaq.qubit):
    rx(x, q)
    h(q)

@cudaq.kernel()
def kernelAdjoint(N : int):
    q = cudaq.qvector(N)
    cudaq.adjoint(rx_and_h_gate, np.pi, q[2])


counts = cudaq.sample(kernelAdjoint, 4)
print(counts)

{ 0010:470 0000:530 }



- #### Example: Hadamard test:

Consider the observable $O$ and two generic quantum states $\ket{\psi}$ and $\ket{\phi}$. We want to calculate the quantity
$$
\bra{\psi} O \ket{\phi}.
$$
where $O$ is a Pauli operator.

First of all we shall prepare the states $\ket{\psi}$ and $\ket{\phi}$ using a quantum circuit for each of them. So we  have
$$
\ket{\psi} = U_{\psi}\ket{0} \qquad \ket{\phi} = U_{\phi}\ket{0}
$$

Let's define an observable we want to use:
$$
O = X_1X_2
$$

Now we can evaluate the matrix element using the following fact:
$$
\bra{\psi}O\ket{\phi} = \bra{0}U_\psi^\dagger O U_\phi\ket{0}
$$
This is just an expectation value which can be solved with a simple Hadamard test. The probability to measure $0$ or $1$ in the ancilla qubit is

$$
P(0) = \frac{1}{2} \left[ I + Re \bra{\psi} O \ket{\phi} \right]
$$

$$
P(1) = \frac{1}{2} \left[ I - Re \bra{\psi} O \ket{\phi} \right]
$$

The difference between the probability of $0$ and $1$ gives 

$$
P(0)-P(1) = Re \bra{\psi} O \ket{\phi}
$$

### A- Classical result as reference:

In [3]:
import cudaq
import numpy as np
from functools import reduce

cudaq.set_target('nvidia')

qubit_num=2

@cudaq.kernel
def psi(num:int):
    q=cudaq.qvector(num)
    h(q[1])
    
@cudaq.kernel
def phi(n:int):
    q=cudaq.qvector(n)
    x(q[0])

psi_state=cudaq.get_state(psi,qubit_num)
print('Psi state: ', psi_state)

phi_state=cudaq.get_state(phi,qubit_num)
print('Phi state: ', phi_state)

ham=cudaq.spin.x(0)*cudaq.spin.x(1)
ham_matrix=ham.to_matrix()
print('hamiltonian: ', np.array(ham_matrix))

exp_val=reduce(np.dot,(np.array(psi_state).conj().T, ham_matrix, phi_state))

print('Numerical expectation value: ', exp_val) 

Psi state:  (0.707107,0)
(0,0)
(0.707107,0)
(0,0)

Phi state:  (0,0)
(1,0)
(0,0)
(0,0)

hamiltonian:  [[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]]
Numerical expectation value:  (0.7071067690849304+0j)


### B- QC result:

In [4]:
import cudaq
import numpy as np

cudaq.set_target('nvidia')

qubit_num=2

@cudaq.kernel
def U_psi(q:cudaq.qview):
    h(q[1])

@cudaq.kernel
def U_phi(q:cudaq.qview):
    x(q[0])

@cudaq.kernel  
def ham_cir(q:cudaq.qview):
    x(q[0])
    x(q[1])

@cudaq.kernel
def kernel(n:int):
    ancilla=cudaq.qubit()
    q=cudaq.qvector(n)
    h(ancilla)
    cudaq.control(U_phi,ancilla,q)
    cudaq.control(ham_cir,ancilla,q)
    cudaq.control(U_psi,ancilla,q)
    
    h(ancilla)
    
    mz(ancilla)
    
shots=50000    
count=cudaq.sample(kernel,qubit_num, shots_count=shots)    
print(count)

mean_val=(count['0']-count['1'])/shots
error=np.sqrt(2*count['0']*count['1']/shots)/shots
print('Observable QC: ', mean_val,'+ -', error)
    

{ 0:42544 1:7456 }

Observable QC:  0.70176 + - 0.002252849090374231


- ### With multi-QPU

In [5]:
import cudaq
import numpy as np

cudaq.set_target("nvidia-mqpu")

qubit_num=2

target = cudaq.get_target()
qpu_count = target.num_qpus()
print("Number of QPUs:", qpu_count)

@cudaq.kernel
def U_psi(q:cudaq.qview, theta:float):
    ry(theta, q[1])

@cudaq.kernel
def U_phi(q:cudaq.qview, theta: float):
    rx(theta, q[0])

@cudaq.kernel  
def ham_cir(q:cudaq.qview):
    x(q[0])
    x(q[1])

@cudaq.kernel
def kernel(n:int, angle:float, theta:float):
    ancilla=cudaq.qubit()
    q=cudaq.qvector(n)
    h(ancilla)
    cudaq.control(U_phi,ancilla,q,theta)
    cudaq.control(ham_cir,ancilla,q)
    cudaq.control(U_psi,ancilla,q, angle)
    
    h(ancilla)
        
    mz(ancilla)
    
shots=50000  
angle=[0.0, 1.5,3.14,0.7]
theta=[0.6, 1.2 ,2.2 ,3.0]

result=[]
for i in range(4):  
    count=cudaq.sample_async(kernel,qubit_num, angle[i], theta[i], shots_count=shots,qpu_id=i)  
    result.append(count)  

mean_val=np.zeros(len(angle))
i=0
for count in result:
    print(i)
    i_result=count.get()
    print(i_result)
    mean_val[i]=(i_result['0']-i_result['1'])/shots
    error=np.sqrt(2*i_result['0']*i_result['1']/shots)/shots
    print('Observable QC: ',  mean_val[i],'+ -', error)
    i+=1

my_mat=np.zeros((2,2),dtype=float)
m=0
for k in range(2):
    for j in range(2):
        my_mat[k,j]=mean_val[m]
        m+=1   

E,V=np.linalg.eigh(my_mat)

print('Compute eigen-values and eigen-vectors')
print('Eigen values: ')
print(E)

print('Eigenvector: ')
print(V)

Number of QPUs: 5
0
{ 0:24941 1:25059 }

Observable QC:  -0.00236 + - 0.0031622688538452894
1
{ 0:25130 1:24870 }

Observable QC:  0.0052 + - 0.0031622349058853926
2
{ 0:24863 1:25137 }

Observable QC:  -0.00548 + - 0.003162230177580374
3
{ 0:25027 1:24973 }

Observable QC:  0.00108 + - 0.0031622758159275104
Compute eigen-values and eigen-vectors
Eigen values: 
[-0.00638359  0.00510359]
Eigenvector: 
[[-0.80605967  0.59183427]
 [-0.59183427 -0.80605967]]


## 2- Pauli word & ``exp_pauli()``:

In [2]:
import cudaq
from cudaq import spin

cudaq.set_target('nvidia')

ham=-0.106477- 0.0454063*spin.x(0)*spin.x(1)*spin.y(2)*spin.y(3) +0.174073*spin.z(2
                )*spin.z(3)+0.0454063*spin.y(0)*spin.x(1)*spin.x(2)*spin.y(3)


@cudaq.kernel
def kernel_pauli_word(theta: float, var: cudaq.pauli_word):
    q = cudaq.qvector(4)
    x(q[0])
    x(q[1])
    exp_pauli(theta, q, var)

exp_val = cudaq.observe(kernel_pauli_word, ham, 0.11, 'XXXY').expectation()

print('Expectation value: ', exp_val)

Expectation value:  0.04777800003152044


In [3]:
import cudaq
from cudaq import spin

cudaq.set_target('nvidia')

ham=-0.106477- 0.0454063*spin.x(0)*spin.x(1)*spin.y(2)*spin.y(3) +0.174073*spin.z(2
                )*spin.z(3)+0.0454063*spin.y(0)*spin.x(1)*spin.x(2)*spin.y(3)


@cudaq.kernel
def kernel_pauli_word_list(theta: float, paulis: list[cudaq.pauli_word]):
    q = cudaq.qvector(4)
    x(q[0])
    x(q[1])
    for p in paulis:
        exp_pauli(theta, q, p)

exp_val = cudaq.observe(kernel_pauli_word_list, ham, 0.11, ['XXXY','XYZY']).expectation()

print('Expectation value: ', exp_val)

Expectation value:  0.04382123044540878


## 3- State handling: qvector initialize and qvector state initialize

- #### qvector initialization

In [14]:
import cudaq

cudaq.set_target('nvidia')

#c = [0. + 0j, 0., 0., 1.]
c = [1. / np.sqrt(2.) + 0j, 0., 0., 1. / np.sqrt(2.)]

@cudaq.kernel
def kernel(vec: list[complex]):
    q = cudaq.qvector(vec)
    x(q.front())
    y(q.back())
    h(q)
    mz(q)

counts = cudaq.sample(kernel, c)

print(counts)

{ 10:511 01:489 }



- #### qvector state initialize

Efficient state vector handling (retrieval and initialization) in simulation is an important requirement for various applications/algorithms

- Dynamical quantum simulation

![img](./state-handl.png)

#### Example: Trotter dynamical simulation:

In [1]:
import cudaq
import time
import numpy as np
from typing import List

spins = 11  
steps = 10  

cudaq.set_target("nvidia")

# Alternating up/down spins
@cudaq.kernel
def getInitState(numSpins: int):
    q = cudaq.qvector(numSpins)
    for qId in range(0, numSpins, 2):
        x(q[qId])


# This performs a single-step Trotter on top of an initial state, e.g.,
# result state of the previous Trotter step.
@cudaq.kernel
def trotter(state: cudaq.State, coefficients: List[complex],
            words: List[cudaq.pauli_word], dt: float):
    q = cudaq.qvector(state)
    for i in range(len(coefficients)):
        exp_pauli(coefficients[i].real * dt, q, words[i])

def run_steps(steps: int, spins: int):
    g = 1.0
    Jx = 1.0
    Jy = 1.0
    Jz = g
    dt = 0.05
    n_steps = steps
    n_spins = spins
    omega = 2 * np.pi

    def heisenbergModelHam(t: float) -> cudaq.SpinOperator:
        tdOp = cudaq.SpinOperator(num_qubits=n_spins)
        for i in range(0, n_spins - 1):
            tdOp += (Jx * cudaq.spin.x(i) * cudaq.spin.x(i + 1))
            tdOp += (Jy * cudaq.spin.y(i) * cudaq.spin.y(i + 1))
            tdOp += (Jz * cudaq.spin.z(i) * cudaq.spin.z(i + 1))
        for i in range(0, n_spins):
            tdOp += (np.cos(omega * t) * cudaq.spin.x(i))
        return tdOp

    # Collect coefficients from a spin operator so we can pass them to a kernel
    def termCoefficients(op: cudaq.SpinOperator) -> List[complex]:
        result = []
        ham.for_each_term(lambda term: result.append(term.get_coefficient()))
        return result

    # Collect Pauli words from a spin operator so we can pass them to a kernel
    def termWords(op: cudaq.SpinOperator) -> List[str]:
        result = []
        ham.for_each_term(lambda term: result.append(term.to_string(False)))
        return result

    # Observe the average magnetization of all spins (<Z>)
    average_magnetization = cudaq.SpinOperator(num_qubits=n_spins)
    for i in range(0, n_spins):
        average_magnetization += ((1.0 / n_spins) * cudaq.spin.z(i))
    average_magnetization -= 1.0

    # Run loop
    state = cudaq.get_state(getInitState, n_spins)
    
    results = []
    times = []
    for i in range(0, n_steps):
        start_time = time.time()
        ham = heisenbergModelHam(i * dt)
        coefficients = termCoefficients(ham)
        words = termWords(ham)
        magnetization_exp_val = cudaq.observe(trotter, average_magnetization,
                                              state, coefficients, words, dt)
        result = magnetization_exp_val.expectation()
        results.append(result)
        state = cudaq.get_state(trotter, state, coefficients, words, dt)
        stepTime = time.time() - start_time
        times.append(stepTime)
        print(f"Step {i}: time [s]: {stepTime}, result: {result}")

    print("")
    # Print all the times and results (useful for plotting).
    print(f"Step times: {times}")
    print(f"Results: {results}")

    print("")

start_time = time.time()
run_steps(steps, spins)
print(f"Total time: {time.time() - start_time}s")

Step 0: time [s]: 0.05071830749511719, result: -0.09042024163828166
Step 1: time [s]: 0.007488250732421875, result: -0.08898564687193886
Step 2: time [s]: 0.008493900299072266, result: -0.08698024360923415
Step 3: time [s]: 0.007264375686645508, result: -0.08507694741170907
Step 4: time [s]: 0.007196187973022461, result: -0.08394118068746997
Step 5: time [s]: 0.008506298065185547, result: -0.08394076573115139
Step 6: time [s]: 0.007228851318359375, result: -0.08502222139504187
Step 7: time [s]: 0.007172107696533203, result: -0.08677832064885871
Step 8: time [s]: 0.00728154182434082, result: -0.08863390649349775
Step 9: time [s]: 0.007157087326049805, result: -0.09005513983609514

Step times: [0.05071830749511719, 0.007488250732421875, 0.008493900299072266, 0.007264375686645508, 0.007196187973022461, 0.008506298065185547, 0.007228851318359375, 0.007172107696533203, 0.00728154182434082, 0.007157087326049805]
Results: [-0.09042024163828166, -0.08898564687193886, -0.08698024360923415, -0.0

## 4- compute_action(): 

- ```compute_action(U,V)```: will invoke $U V U^\dagger$. $U$ is the compute block and $V$ is the action block.

- Example: Grover algorithm: 

![img](./grover-circuit.png)

In [6]:
import cudaq
from typing import Callable

cudaq.set_target('nvidia')

@cudaq.kernel
def reflect(qubits: cudaq.qview):
    ctrls = qubits.front(qubits.size() - 1)
    last = qubits.back()
    cudaq.compute_action(lambda: (h(qubits), x(qubits)),
                          lambda: z.ctrl(ctrls, last))

@cudaq.kernel
def oracle(q: cudaq.qview):
    z.ctrl(q[0], q[2])
    z.ctrl(q[1], q[2])

@cudaq.kernel
def grover(N: int, M: int, oracle: Callable[[cudaq.qview], None]):
    q = cudaq.qvector(N)
    h(q)
    for i in range(M):
        oracle(q)
        reflect(q)
    mz(q)

counts = cudaq.sample(grover, 3, 1, oracle)
print(counts)

{ 101:484 011:516 }

